# Data Collection and Cleaning
---

## Table of Contents
 #### 1.  [Getting data from the REST API](#getting_data)
 #### 2.  [Creating the intakes Data Frame](#intakes)
 #### 3.  [Creating the outcomes Data Frame](#outcomes)
 #### 4.  [Joining together](#joining)
 #### 5.  [Exporting the cleaned data](#exporting)
 ---

## 1. Getting data from the REST API <a id="getting_data"></a>

In [1]:
import numpy as np
import pandas as pd
import requests
from pandas.io.json import json_normalize
from sodapy import Socrata
import warnings
warnings.filterwarnings('ignore')

In [2]:
client = Socrata("data.austintexas.gov", None)
results = client.get("fdzn-9yqv", limit=100000)

## 2. Creating the intakes Data Frame <a id="intakes"></a>

In [3]:
intake_df = pd.DataFrame.from_records(results)
intake_df = intake_df.set_index("animal_id")

#### I. Rename datetime and sex to normal names


In [4]:
#### I. Rename datetime and sex to normal names
intake_df.drop(['datetime2'], axis=1, inplace=True)
intake_df.rename(columns={'datetime': 'date_in', 'sex_upon_intake': 'sex'}, inplace=True)

#### II. Remove other animals except dogs


In [5]:
intake_df = intake_df.loc[intake_df['animal_type'] == "Dog"]

#### III. Convert `color` to a list


In [6]:
intake_df.color = intake_df.color.str.split('/')

#### IV. Standardize addresses to use later


In [7]:
intake_df.found_location = intake_df['found_location'].str[:-5].str.replace(" in ", " ")

#### V. Create two separate columns, `fixed` and `sex` from `sex`


In [8]:
sex_series = intake_df.sex.str.split(" ")
intake_df['fixed'] = sex_series.str[0]
intake_df['sex'] = sex_series.str[1]
intake_df.fixed = intake_df.fixed.map({
    "Neutered": "Yes",
    "Spayed": "Yes",
    "Intact": "No"
})

#### VI. Clean up the dogs' names


In [9]:
intake_df.name = intake_df.name.str.replace("*", "")

#### VII. Convert `date_in` to `datetime` object


In [10]:
intake_df.date_in = intake_df.date_in.apply(pd.to_datetime)

#### VIII. Convert `age_upon_intake` to years


In [11]:
def fix_age_intake(date):
    years = 0.0
    if ("year" in date[1]):
        years =  float(date[0])
        
    elif ("month" in date[1]):
        years =  (float(date[0]) / 12)
        
    elif ("week" in date[1]):
        years =  (float(date[0]) / 52)
        
    elif ("day" in date[1]):
        years =  (float(date[0]) / 365)
        
    return years

In [12]:
intake_df['age_upon_intake'] = intake_df.age_upon_intake.str.split().apply(fix_age_intake)

## 3. Creating the outcomes Data Frame <a id="outcomes"></a>

In [13]:
client = Socrata("data.austintexas.gov", None)
results = client.get("9t4d-g238", limit=100000)

outcomes_df = pd.DataFrame.from_records(results)
outcomes_df = outcomes_df.set_index("animal_id")

#### I. Rename and convert `datetime` to `date_out` as a `datetime` object

In [14]:
outcomes_df.rename(columns={'datetime': 'date_out'}, inplace=True)

In [15]:
outcomes_df.date_out = outcomes_df.date_out.apply(pd.to_datetime)

#### II. Remove unnecessary columns from `outcomes_df`

In [16]:
outcomes_df = outcomes_df[['date_of_birth', 'date_out', "outcome_subtype", "outcome_type"]]

## 4. Joining together <a id="joining"></a>

#### I. Outer merge `intake_df` and `outcomes_df`

In [17]:
combined_df = intake_df.merge(outcomes_df, on="animal_id", how="outer")

#### II. Drop incorrectly entered rows (rows that don't have `date_in`)

In [18]:
combined_df = combined_df.dropna(axis=0, subset=['date_in'])

#### III. Create column `in_shelter` if `outcome_type` has a value

In [19]:
combined_df['in_shelter'] = "No"
combined_df['in_shelter'][combined_df.outcome_type.isnull()] = "Yes"

#### IV. Create column `time_in_shelter` and remove invalid rows (negative times)

In [20]:
combined_df['time_in_shelter'] = combined_df.date_out - combined_df.date_in

In [21]:
mask = ((combined_df.time_in_shelter < pd.Timedelta(0)))
combined_df = combined_df.drop(mask.loc[mask == True].index)

#### VI. Create `age_in` and `age_out` from the dog's birthday

In [22]:
combined_df.date_of_birth = combined_df.date_of_birth.apply(pd.to_datetime)

In [23]:
combined_df["age_in"] = combined_df.date_in - combined_df.date_of_birth
combined_df.age_in = (combined_df.age_in.apply(lambda x: x.days)/365)

In [24]:
# if age in didn't work, then replace it with age_upon_intake
combined_df.age_in = np.where(combined_df.age_in.isnull(), combined_df.age_upon_intake, combined_df.age_in)

In [25]:
combined_df["age_out"] = combined_df.date_out - combined_df.date_of_birth
combined_df.age_out =  (combined_df.age_out.apply(lambda x: x.days)/365)

#### VII. Remove duplicate rows (from user entry error)

In [26]:
features = combined_df.columns.tolist()
features.remove("color")
features.remove("date_out")
features.remove("outcome_type")
combined_df = combined_df.drop_duplicates(subset=features)

In [27]:
combined_df.drop(['age_upon_intake', 'animal_type'], axis=1, inplace=True)

#### VIII. Create a new Data Frame without repeat-admit dogs

In [28]:
combined_unique_df = combined_df[~combined_df.index.duplicated(keep='first')]

**Create Column that indicates if a dog is a purebred or mixed**

In [29]:
combined_unique_df['is_mixed'] = (combined_unique_df.breed.str.contains("Mix") | combined_unique_df.breed.str.contains("/"))

In [30]:
combined_unique_df['is_mixed'] = combined_unique_df['is_mixed'].map({
    True: 1,
    False: 0
})

In [31]:
# remove all breeds with / in it
#combined_unique_df = combined_unique_df.loc[~combined_unique_df.breed.str.contains('/')]
#combined_unique_df.breed.value_counts()

**Create a column that indicates is a dog has a name**

In [32]:
combined_unique_df['has_name']= combined_unique_df.name.notnull()
combined_unique_df['has_name'] = combined_unique_df['has_name'].map({
    True: 1,
    False: 0
})

In [33]:
combined_unique_df

,breed,color,date_in,found_location,intake_condition,intake_type,name,sex,fixed,date_of_birth,date_out,outcome_subtype,outcome_type,in_shelter,time_in_shelter,age_in,age_out,is_mixed,has_name
animal_id,,,,,,,,,,,,,,,,,,,
A786884,Beagle Mix,[Tricolor],2019-01-03 16:19:00,2501 Magin Meadow Dr Austin,Normal,Stray,Brock,Male,Yes,2017-01-03,2019-01-08 15:11:00,Partner,Transfer,No,4 days 22:52:00,2.000000,2.013699,1,1
A706918,English Springer Spaniel,"[White, Liver]",2015-07-05 12:59:00,9409 Bluegrass Dr Austin,Normal,Stray,Belle,Female,Yes,2007-07-05,2015-07-05 15:13:00,NaN,Return to Owner,No,0 days 02:14:00,8.005479,8.005479,0,1
A724273,Basenji Mix,"[Sable, White]",2016-04-14 18:43:00,2818 Palomino Trail Austin,Normal,Stray,Runster,Male,No,2015-04-17,2016-04-21 17:17:00,NaN,Return to Owner,No,6 days 22:34:00,0.994521,1.013699,1,1
A778404,German Shepherd Mix,"[Black, Tan]",2018-08-14 10:19:00,Austin,Normal,Owner Surrender,Max,Male,No,2014-08-14,2018-08-17 18:03:00,NaN,Adoption,No,3 days 07:44:00,4.002740,4.010959,1,1
A682524,Doberman Pinsch/Australian Cattle Dog,"[Tan, Gray]",2014-06-29 10:38:00,800 Grove Blvd Austin,Normal,Stray,Rio,Male,Yes,2010-06-29,2014-07-02 14:16:00,NaN,Return to Owner,No,3 days 03:38:00,4.002740,4.010959,1,1
A743852,Labrador Retriever Mix,[Chocolate],2017-02-18 12:46:00,Austin,Normal,Owner Surrender,Odin,Male,Yes,2015-02-18,2017-02-21 17:44:00,NaN,Return to Owner,No,3 days 04:58:00,2.002740,2.010959,1,1
A708452,Labrador Retriever Mix,"[Black, White]",2015-07-30 14:37:00,Austin,Normal,Public Assist,Mumble,Male,No,2013-07-28,2015-08-04 18:17:00,NaN,Return to Owner,No,5 days 03:40:00,2.005479,2.019178,1,1
A760053,Chihuahua Shorthair,"[White, Tan]",2017-10-11 15:46:00,8800 South First Street Austin,Normal,Stray,NaN,Male,No,2015-10-11,2017-10-15 18:30:00,NaN,Adoption,No,4 days 02:44:00,2.002740,2.013699,0,0
A707375,Pit Bull,"[Brown, White]",2015-07-11 18:19:00,Galilee Court And Damita Jo Dr Manor,Normal,Stray,Candy Cane,Female,No,2015-01-11,2015-11-28 17:31:00,NaN,Adoption,No,139 days 23:12:00,0.495890,0.879452,0,1


## 5. Exporting the cleaned data <a id="exporting"></a>

In [34]:
combined_df.to_csv('../data/austin_shelter.csv')

In [35]:
combined_unique_df.to_csv('../data/unique_austin_shelter.csv')

---

### Get geocodes from addresses

*NOTE: DO NOT RUN THESE CELLS*